In [1]:
import pandas as pd

In [2]:
!pip install sentence_transformers
import pandas as pd
from sentence_transformers import SentenceTransformer #사전학습 모델을 가져오는 것
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('jhgan/ko-sroberta-multitask')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
sentences=["안녕하세요?", "테스트를 위한 문장입니다."]
embeddings=model.encode(sentences)
print(embeddings)

[[-0.37510455 -0.7733843   0.5927708  ...  0.579235    0.32683495
  -0.6508965 ]
 [-0.17084251 -0.61791253 -0.5370927  ...  0.37742916  0.75107646
  -0.19669956]]


In [20]:
df=pd.read_csv("/content/drive/MyDrive/질환증상크롤링 (1).csv")

In [21]:
df=df.fillna('-')

In [22]:
df['input'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [23]:
import re
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r",","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()

    return s

df['input']= df['input'].apply(lambda x : normalize_text(x))

In [24]:
df['embedding']=pd.Series([[]]*len(df)) # 길이를 맞춰주기 위해서 곱함
df['embedding']=df['input'].map(lambda x : list(model.encode(x)))
df.head()

,부위,질환명,정의,원인,증상,관련질환,진료과,input,embedding
0,배,간 전이(Liver Metastasis),간 전이는 간 이외의 다른 부위에서 생긴 암세포가 간으로 퍼진 악성 종양을 의미합니다.,간은 다른 장기에 비해 크고 혈액 공급이 풍부하여 여과 작용을 합니다. 따라서 다른...,"간 전이의 경우 원발성 암의 증상만 나타나는 경우가 많습니다. 발열, 발한, 무기력...","대장암, 췌장암, 식도암, 위암, 폐암, 유방암, 악성 흑색종",종양내과,배 간 전이(Liver Metastasis) 간 전이는 간 이외의 다른 부위에서 생...,"[-0.07185317, 0.097094, -0.23905303, 0.0953968..."
1,배,간 혈관종(Hepatic hemangioma),"혈관종이란 혈관 조직이 비정상적으로 증식, 확장하여 뭉쳐서 덩어리를 이룬 것입니다....","간 혈관종은 간에 발생하는 양성 종양 가운데 가장 흔한 것이지만, 그 원인은 알려져...",간 혈관종은 대부분이 작고 증상이 없어 모르고 지나가는 경우가 많습니다. 최근에 건...,간의 양성 신생물,소화기내과,배 간 혈관종(Hepatic hemangioma) 혈관종이란 혈관 조직이 비정상적으...,"[-0.5189258, -0.49328864, -0.60605323, 0.13974..."
2,배,간경화(Liver cirrhosis),간경화는 학술적 병명인 '간경변증'의 일반화된 명칭입니다. 간세포 손상(간염)이 장...,우리나라에서 발병하는 간경변증의 원인으로는 만성 B형 간염이 약 70% 정도로 가장...,간경변증은 초기에 대부분 증상이 없습니다. 간경변증이 많이 진행한 이후부터 증상이 ...,"간세포암, 자가면역성 간염, 만성 B형 간염, 만성 C형 간염, 간염, 간성 뇌증,...",소화기내과,배 간경화(Liver cirrhosis) 간경화는 학술적 병명인 '간경변증'의 일반...,"[-0.5106691, -0.1965011, -0.5117235, 0.1494586..."
3,배,간내 담석(IHD stone),간내 담석은 간 내부의 담도에 결석이 생기는 것을 의미합니다. 색소성 담석이 주를 ...,간내 담석은 간 우엽보다 간 좌엽에서 더 많이 발생합니다. 이는 좌엽의 담도가 우엽...,단순한 간내 담석만 있는 경우 담석이 담도를 막기 전까지는 증상이 잘 나타나지 않습...,"담관암, 간세포암, 담석",소화기내과,배 간내 담석(IHD stone) 간내 담석은 간 내부의 담도에 결석이 생기는 것을...,"[-0.46888924, -0.25883108, 0.07719532, 0.37983..."
4,배,간농양(Liver abscess),간농양은 세균이나 기생충에 의해 간 안에 고름집(농양)이 생기는 것을 의미합니다.,정상인의 간에는 간농양이 잘 생기지 않습니다. 다만 간에서 담즙이 배출되는 담관이 ...,"화농성 간농양은 미열 및 오한, 상복부 통증, 오심(구역질), 구토, 설사, 무기력...",아메바성 간농양,감염내과,배 간농양(Liver abscess) 간농양은 세균이나 기생충에 의해 간 안에 고름...,"[-0.44751146, -0.3565954, -0.04824531, 0.47608..."


In [25]:
test_text='기침, 가래, 호흡 곤란과 같은 증상'
ebd=model.encode(test_text)

In [26]:
df['distance']=df['embedding'].map(lambda x : cosine_similarity([ebd], [x]).squeeze())#차원을 없애주기 위해서 squeeze를 씀.
df.head()

,부위,질환명,정의,원인,증상,관련질환,진료과,input,embedding,distance
0,배,간 전이(Liver Metastasis),간 전이는 간 이외의 다른 부위에서 생긴 암세포가 간으로 퍼진 악성 종양을 의미합니다.,간은 다른 장기에 비해 크고 혈액 공급이 풍부하여 여과 작용을 합니다. 따라서 다른...,"간 전이의 경우 원발성 암의 증상만 나타나는 경우가 많습니다. 발열, 발한, 무기력...","대장암, 췌장암, 식도암, 위암, 폐암, 유방암, 악성 흑색종",종양내과,배 간 전이(Liver Metastasis) 간 전이는 간 이외의 다른 부위에서 생...,"[-0.07185317, 0.097094, -0.23905303, 0.0953968...",0.354859
1,배,간 혈관종(Hepatic hemangioma),"혈관종이란 혈관 조직이 비정상적으로 증식, 확장하여 뭉쳐서 덩어리를 이룬 것입니다....","간 혈관종은 간에 발생하는 양성 종양 가운데 가장 흔한 것이지만, 그 원인은 알려져...",간 혈관종은 대부분이 작고 증상이 없어 모르고 지나가는 경우가 많습니다. 최근에 건...,간의 양성 신생물,소화기내과,배 간 혈관종(Hepatic hemangioma) 혈관종이란 혈관 조직이 비정상적으...,"[-0.5189258, -0.49328864, -0.60605323, 0.13974...",0.316960
2,배,간경화(Liver cirrhosis),간경화는 학술적 병명인 '간경변증'의 일반화된 명칭입니다. 간세포 손상(간염)이 장...,우리나라에서 발병하는 간경변증의 원인으로는 만성 B형 간염이 약 70% 정도로 가장...,간경변증은 초기에 대부분 증상이 없습니다. 간경변증이 많이 진행한 이후부터 증상이 ...,"간세포암, 자가면역성 간염, 만성 B형 간염, 만성 C형 간염, 간염, 간성 뇌증,...",소화기내과,배 간경화(Liver cirrhosis) 간경화는 학술적 병명인 '간경변증'의 일반...,"[-0.5106691, -0.1965011, -0.5117235, 0.1494586...",0.377894
3,배,간내 담석(IHD stone),간내 담석은 간 내부의 담도에 결석이 생기는 것을 의미합니다. 색소성 담석이 주를 ...,간내 담석은 간 우엽보다 간 좌엽에서 더 많이 발생합니다. 이는 좌엽의 담도가 우엽...,단순한 간내 담석만 있는 경우 담석이 담도를 막기 전까지는 증상이 잘 나타나지 않습...,"담관암, 간세포암, 담석",소화기내과,배 간내 담석(IHD stone) 간내 담석은 간 내부의 담도에 결석이 생기는 것을...,"[-0.46888924, -0.25883108, 0.07719532, 0.37983...",0.307250
4,배,간농양(Liver abscess),간농양은 세균이나 기생충에 의해 간 안에 고름집(농양)이 생기는 것을 의미합니다.,정상인의 간에는 간농양이 잘 생기지 않습니다. 다만 간에서 담즙이 배출되는 담관이 ...,"화농성 간농양은 미열 및 오한, 상복부 통증, 오심(구역질), 구토, 설사, 무기력...",아메바성 간농양,감염내과,배 간농양(Liver abscess) 간농양은 세균이나 기생충에 의해 간 안에 고름...,"[-0.44751146, -0.3565954, -0.04824531, 0.47608...",0.484501


In [27]:
answer1 = df.loc[df['distance'].idxmax()]

print('챗봇 답변 :', answer1['input'])
print('유사도 :', answer1['distance'])


챗봇 답변 : 가슴 객혈(Hemoptysis) 객혈은 기침과 함께 혈액이 섞인 가래가 나오는 것을 의미합니다. 기침할 때 흉통을 동반하는 경우가 많으며 피는 대부분 선홍빛입니다. 객혈은 전체적인 양보다 속도가 중요합니다. 시간당 150mL 정도의 속도로 피가 나온다면 질식될 위험이 있습니다. 적은 양의 객혈이 반복된다면 폐암 기관지확장증 승모판협착증 반복성 폐경색증 등의 가능성이 있습니다. 고름 성분이 많은 객담이 객혈과 함께 나오는 경우 폐렴 기관지확장증 폐농양 등의 가능성이 높습니다. 객혈의 양이 많은 경우 폐결핵 괴사성 폐렴 기관지확장증 폐암 등을 의심해 볼 수 있습니다. 그러나 증상의 약 30% 정도는 원인을 찾을 수 없습니다. 객혈의 증상은 가래에 혈액이 묻어 나오는 경우부터 순수한 혈액만 나오는 경우까지 다양합니다. 하루에 100~600mL 이상의 객혈이 나오는 것을 대량 객혈이라고 부르며 이때는 응급조치를 받아야 합니다. 폐 혈전색전증 식도암 폐암 비소세포성 폐암 소세포성 폐암 폐렴 속립성 결핵 기관지확장증 폐결핵 폐 전이 승모판 협착 아스페르길루스증 만성 기관지염 폐부종 결핵 호흡기내과
유사도 : 0.7656356692314148


In [28]:
!pip install openai

In [37]:
from openai import ChatCompletion
from openai import OpenAI
import openai

# OpenAI API 키 설정

client=openai.OpenAI(api_key = "key")

# 상위 유사도 단락 기반 추가 답변 생성 함수
def generate_additional_response(question, context):
    messages = [
        {"role": "system", "content": "다음 내용을 참조하여 사용자의 질문에 간단히 답변줄 것. 문서와 질문에 대한 요약(summaries)이 주어지면 참조('SOURCES')를 사용하여 최종 답변을 작성할 것. 답을 모르면 '모른다'고 말하고 답을 만들어 내려고 하지 말것."},
        {"role": "user", "content": f"SOURCES: {context}\n\n{question}"}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=1000
    )

    return response.choices[0].message.content

# 예제 질문
example_question = "무릎의 통증입니다. 특히 계단을 오르내릴 때 통증이 더 심합니다. 통증은 대개 오전보다 오후에 더 심합니다."

# 유사도 기반 가장 유사한 단락을 context로 사용
context = answer1['input']
additional_response = generate_additional_response(example_question, context)

# 생성된 추가 답변 출력
print("추가 생성된 답변:", additional_response)

추가 생성된 답변: 통증이 계단을 오르내릴 때 더 심해지고 대개 오전보다 오후에 더 심하다면 슬개골연골손상이 의심됩니다. 해당 증상은 슬개골연골손상으로 인한 것으로 보입니다. 종양, 골절 및 관절염도 고려해볼 수 있습니다. 정확한 진단을 위해 의사 상담이 필요합니다.
